## Necessary packages

In [ ]:
import time
import json
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Login

In [ ]:
def login(username, password, seconds_2_wait = 2):
    twitter_login_url = "https://twitter.com/login"
    driver = webdriver.Chrome(service=Service(path_to_chrome_binary), options=options) 
    driver.get(twitter_login_url)        
    time.sleep(seconds_2_wait)

    user = driver.find_element(By.XPATH, "//input")
    user.send_keys(username)
    user.send_keys(Keys.ENTER)        
    time.sleep(seconds_2_wait)  
    
    pw = driver.find_element(By.XPATH, "//input[@autocomplete='current-password']")
    pw.send_keys(password)
    pw.send_keys(Keys.RETURN)    
    time.sleep(seconds_2_wait)
    
    driver.maximize_window()
    driver.refresh()
    time.sleep(seconds_2_wait)
    return driver


def perform_query(driver, query):    
    search_field = driver.find_element(By.XPATH, "//input[@enterkeyhint='search']")
    search_field.click()
    search_field.send_keys(query)
    search_field.send_keys(Keys.RETURN)
    
    
def extract_tweets(driver, verbose=False):    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    articles = soup.find_all("article")
    tweets = []
    for article in articles:
        div = article.find("div", attrs={"data-testid":"tweetText"})
        img = article.find("img", attrs={"alt":"Image", "draggable":"true"})
        tweet = {"tweet_text":div.get_text(), "image_url": "" if img is None else img["src"]}
        tweets.append(json.dumps(tweet))
        
        if verbose:
            print(f"Tweet text:\n{tweet['tweet_text']}")
            if img is not None:
                print(f"Image:\n{tweet['image_url']}") 
            print('\n=============================================================================\n')
    return tweets
    
    
def scroll_down(driver, scroll_threshold=3, sleep_threshold=3, verbose=False): 
    tweets = []    
    for i in tqdm(range(scroll_threshold + 1)):
        # Pull tweets..
        tweets.extend(extract_tweets(driver, verbose=verbose))
        temp = driver.find_element(By.TAG_NAME, "body")
        temp.send_keys(Keys.END)
        time.sleep(sleep_threshold) 
    return tweets

## Setup driver

In [ ]:
path_to_chrome_binary = PATH_TO_YOUR_CHROMEDRIVER # Download here: https://chromedriver.chromium.org/downloads
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0"

options = webdriver.ChromeOptions() 
options.add_argument(f"user-agent=[{user_agent}]")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("detach", True)

## Credentials

In [ ]:
username = YOUR_X_USERNAME
password = YOUR_X_PASSWORD

## Start Crawling

In [ ]:
keyword = "#Holidays"
scroll_threshold = 3

driver = login(username, password)    
perform_query(driver, keyword)

results = scroll_down(driver, scroll_threshold=scroll_threshold)

## Print results

In [ ]:
print(f"Total crawled tweets: {len(results)}") 

In [ ]:
for result in results:
    print(result)
    print()